In [ ]:
# 1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
# 2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
# 3.  You cannot use DropOut layers.
# 4.  You MUST use Image Augmentation Techniques.
# 5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
# 6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
# 7.  You cannot use test images for training the model.
# 8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
# 9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
# 10. You cannot have more than 1 Million parameters in total
# 11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
# 12. You can use any optimization algorithm you need. 
# 13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Concatenate, SeparableConv2D
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [31]:


# Hyperparameters
batch_size = 32
num_classes = 10
epochs = 10
l = 12
num_filter = 18
compression = 1
dropout_rate = 0

In [32]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

In [33]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255  

print(X_train.shape)
print(X_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


# Image Augmentation

In [6]:
aug = ImageDataGenerator(shear_range=0.2,
                         zoom_range=0.2,
                         height_shift_range=0.1,
                         horizontal_flip=True) 

# Model Building and Training

In [45]:
# Dense Block
def denseblock(input, num_filter = 48, dropout_rate = 0.0):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 48, dropout_rate = 0.0):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [46]:
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [47]:
model = Model(inputs=[input], outputs=[output])

In [48]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_312 (Conv2D)             (None, 32, 32, 18)   486         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_312 (BatchN (None, 32, 32, 18)   72          conv2d_312[0][0]                 
__________________________________________________________________________________________________
activation_312 (Activation)     (None, 32, 32, 18)   0           batch_normalization_312[0][0]    
____________________________________________________________________________________________

In [21]:
print(len(model.layers))


263


In [51]:
aug = ImageDataGenerator(shear_range=0.2,
                         zoom_range=0.2,
                         height_shift_range=0.1,
                         horizontal_flip=True) 

# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [52]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="models/CNN_CIFR_APR_21_{epoch:04d}.hdf5", 
    verbose=1, 
    save_weights_only=True,
    period=10)


res_model = model.fit(
                    x = aug.flow(X_train, y_train, batch_size=batch_size,),
                    epochs=150,
                    verbose=1, 
                    validation_data=(X_test, y_test),
                    callbacks = [cp_callback])

Epoch 1/150
1563/1563 [==============================] - 73s 44ms/step - loss: 2.0559 - accuracy: 0.2771 - val_loss: 1.4818 - val_accuracy: 0.4724
Epoch 2/150
1563/1563 [==============================] - 67s 43ms/step - loss: 1.3637 - accuracy: 0.5087 - val_loss: 1.3150 - val_accuracy: 0.5554
Epoch 3/150
1563/1563 [==============================] - 67s 43ms/step - loss: 1.0332 - accuracy: 0.6286 - val_loss: 1.2683 - val_accuracy: 0.5775
Epoch 4/150
1563/1563 [==============================] - 66s 43ms/step - loss: 0.8787 - accuracy: 0.6897 - val_loss: 1.5207 - val_accuracy: 0.5580
Epoch 5/150
1563/1563 [==============================] - 67s 43ms/step - loss: 0.7662 - accuracy: 0.7294 - val_loss: 1.2011 - val_accuracy: 0.6211
Epoch 6/150
1563/1563 [==============================] - 66s 42ms/step - loss: 0.6816 - accuracy: 0.7630 - val_loss: 0.9039 - val_accuracy: 0.7009
Epoch 7/150
1563/1563 [==============================] - 66s 43ms/step - loss: 0.6421 - accuracy: 0.7774 - val_loss: 0

In [57]:
# Continue Trainin from 150th epoch + train for 50 more epochs

model.load_weights("/content/CNN_CIFR_APR_21_0150.hdf5")
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="models/CNN_CIFR_APR_21_{epoch:04d}.hdf5", 
    verbose=1, 
    save_weights_only=True,
    period=10)


res_model = model.fit(
                    x = aug.flow(X_train, y_train, batch_size=batch_size,),
                    epochs=20,
                    verbose=1, 
                    validation_data=(X_test, y_test),
                    callbacks = [cp_callback])

Epoch 1/20
1563/1563 [==============================] - 71s 43ms/step - loss: 0.0626 - accuracy: 0.9779 - val_loss: 0.3811 - val_accuracy: 0.9143
Epoch 2/20
1563/1563 [==============================] - 67s 43ms/step - loss: 0.0453 - accuracy: 0.9852 - val_loss: 0.3894 - val_accuracy: 0.9139
Epoch 3/20
1563/1563 [==============================] - 66s 43ms/step - loss: 0.0392 - accuracy: 0.9858 - val_loss: 0.3807 - val_accuracy: 0.9155
Epoch 4/20
1563/1563 [==============================] - 67s 43ms/step - loss: 0.0362 - accuracy: 0.9869 - val_loss: 0.3820 - val_accuracy: 0.9192
Epoch 5/20
1563/1563 [==============================] - 67s 43ms/step - loss: 0.0363 - accuracy: 0.9873 - val_loss: 0.3781 - val_accuracy: 0.9206
Epoch 6/20
1563/1563 [==============================] - 67s 43ms/step - loss: 0.0321 - accuracy: 0.9885 - val_loss: 0.3949 - val_accuracy: 0.9164
Epoch 7/20
1563/1563 [==============================] - 67s 43ms/step - loss: 0.0314 - accuracy: 0.9889 - val_loss: 0.3933 -

In [60]:
# Test the model
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 3s 11ms/step - loss: 0.4300 - accuracy: 0.9200
Test loss: 0.4300319254398346
Test accuracy: 0.9200000166893005


In [59]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk
